## Повышение отзывчивости системы


## Кэширование данных
Самый лучший способ ускорить вычисления – не совершать их вообще.
Эффективность кэширования базируется на **Принципе Парето (80/20)**: 20% горячих данных генерируют 80% трафика.

### Где кэшировать?
1.  **Браузер**: Кэширование статики (CSS, JS).
2.  **CDN**: Кэширование контента ближе к пользователю.
3.  **Load Balancer / Reverse Proxy**: Кэширование ответов API.
4.  **Service**: In-memory cache (локальный).
5.  **Database**: Буферы БД.

### Инвалидация кэша (Cache Invalidation)
"Есть только две сложные вещи в CS: инвалидация кэша и именование переменных".

**Стратегии:**
1.  **Write-through (Сквозная запись)**: Пишем в Кэш + в БД одновременно. Медленная запись, зато данные всегда консистентны.
2.  **Write-around (Запись в обход)**: Пишем в БД. Кэш промахивается и подтягивает данные.
3.  **Write-back (Реверсивная запись)**: Пишем в Кэш (быстро). Фоном сбрасываем в БД. Риск потери данных!

### Вытеснение данных (Eviction Policies)
*   **LRU (Least Recently Used)**: Удаляем те, что давно не запрашивали (Самый популярный).
*   **LFU (Least Frequently Used)**: Удаляем самые редкие.
*   **FIFO**: Кто первый пришел, тот первый ушел.

---

## Индексирование БД
Ускоряет поиск (Read), но замедляет запись (Write).
*   **B-Tree**: Стандарт для большинства индексов.
*   **LSM-Tree**: Для высокой скорости записи (Cassandra).

### Генерация ID (Unique IDs)
Как генерировать уникальные ID в распределенной системе?
1.  **UUID (GUID)**: 128 бит. Генерируется независимо. Минус: длинный, не сортируется.
2.  **Ticket Server**: Централизованный сервис (Flickr). Минус: точка отказа.
3.  **Twitter Snowflake**: 64 бита.
    *   41 бит: Timestamp.
    *   10 бит: Machine ID.
    *   12 бит: Sequence number.
    *   *Плюс*: Сортируется по времени, компактный.

---

## Протоколы соединений (Client-Server)
Как получать обновления без F5?
1.  **AJAX Polling**: Клиент постоянно спрашивает "Есть что нового?". (Много мусора).
2.  **Long-Polling**: Клиент спрашивает, Сервер "висит" и не отвечает, пока не появятся данные. (Лучше).
3.  **WebSockets**: Постоянный двусторонний канал (TCP). Идеально для чатов/игр.
4.  **Server-Sent Events (SSE)**: Односторонний канал Сервер -> Клиент. (Для новостных лент).


## Практика: LRU Cache (Least Recently Used)


### Как это работает?
Мы храним данные в кэше ограниченного размера.
1. Если данные запрашиваются, они перемещаются в "голову" списка (становятся "свежими").
2. Если кэш переполнен, мы удаляем данные с "хвоста" списка (самые давно неиспользованные).

Используем `OrderedDict`, который помнит порядок вставки.


In [ ]:

from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity: int):
        self.cache = OrderedDict()
        self.capacity = capacity

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        # Перемещаем ключ в конец (помечаем как недавно использованный)
        self.cache.move_to_end(key)
        return self.cache[key]

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = value
        
        # Если превысили размер, удаляем первый элемент (самый старый)
        if len(self.cache) > self.capacity:
            oldest = self.cache.popitem(last=False)
            print(f"🗑️ Вытеснен ключ: {oldest[0]}")

# --- Тест ---
lru = LRUCache(3)
print("Добавляем 1, 2, 3")
lru.put(1, 100)
lru.put(2, 200)
lru.put(3, 300)
print(f"Кэш: {list(lru.cache.keys())}")

print("Читаем 1 (теперь он стал 'свежим')")
lru.get(1)
print(f"Кэш: {list(lru.cache.keys())}")

print("Добавляем 4 (должен вытеснить 2, т.к. 1 мы только что трогали, а 2 - самый старый)")
lru.put(4, 400)
print(f"Кэш: {list(lru.cache.keys())}")


## Практика: Twitter Snowflake ID


### Как генерировать уникальные ID?
Битовые сдвиги позволяют упаковать Timestamp, ID машины и Порядковый номер в одно 64-битное число.
Структура (64 бита):
*   1 бит: знак (не исп.)
*   41 бит: Timestamp (мс)
*   10 бит: Machine ID
*   12 бит: Sequence (счетчик внутри одной миллисекунды)


In [ ]:

import time

class SnowflakeGenerator:
    def __init__(self, machine_id):
        self.machine_id = machine_id
        self.sequence = 0
        self.last_timestamp = -1
        
        # Константы (битовые сдвиги)
        self.machine_id_bits = 10
        self.sequence_bits = 12
        
        self.machine_id_shift = self.sequence_bits
        self.timestamp_shift = self.sequence_bits + self.machine_id_bits
        
        self.sequence_mask = (1 << self.sequence_bits) - 1
        
        # Эпоха (начало отсчета, например 2023-01-01)
        self.epoch = 1672531200000

    def _current_timestamp(self):
        return int(time.time() * 1000)

    def next_id(self):
        timestamp = self._current_timestamp()

        if timestamp < self.last_timestamp:
            raise Exception("Часы пошли назад!")

        if timestamp == self.last_timestamp:
            # Если в ту же миллисекунду - увеличиваем счетчик
            self.sequence = (self.sequence + 1) & self.sequence_mask
            if self.sequence == 0:
                # Счетчик переполнился, ждем следующую мс
                while timestamp <= self.last_timestamp:
                    timestamp = self._current_timestamp()
        else:
            # Новая миллисекунда - сбрасываем счетчик
            self.sequence = 0

        self.last_timestamp = timestamp

        # Собираем ID битовыми операциями
        # | TIMESTAMP | MACHINE_ID | SEQUENCE |
        return ((timestamp - self.epoch) << self.timestamp_shift) |                (self.machine_id << self.machine_id_shift) |                self.sequence

# --- Тест ---
gen = SnowflakeGenerator(machine_id=1)
print(f"Machine ID: 1")
for _ in range(5):
    uid = gen.next_id()
    print(f"Generated ID: {uid} (bin: {bin(uid)})")
